In [1]:
%load_ext tensorboard

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from keras_visualizer import visualizer
from IPython.display import Image
import time
import datetime

In [3]:
tf.__version__

'2.5.2'

In [4]:
### ignore TensorFlow INFO messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [5]:
HIDDEN_LAYER_1 = 16
HIDDEN_LAYER_2 = 128

model = tf.keras.Sequential()
model.add(layers.Dense(HIDDEN_LAYER_1, input_dim=2, activation='sigmoid'))

if HIDDEN_LAYER_2 > 0:
    model.add(layers.Dense(HIDDEN_LAYER_2, activation='sigmoid'))

model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2176      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,353
Trainable params: 2,353
Non-trainable params: 0
_________________________________________________________________


In [7]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), "%d_%d" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [8]:
# dot muss im env/bin Verzeichnis sein, beim Starten des Jupyter Notebooks
visualizer(model, format='svg', view=False)
Image(url=f'graph.svg?{str(int(time.time()))}')

In [9]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [10]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [11]:
epochs = 750
if HIDDEN_LAYER_2 > 0:
    epochs = 1500

t1 = time.time()
model.fit(training_data,
          target_data, 
          callbacks=[tensorboard_callback], 
          epochs=epochs)
t2 = time.time()

Epoch 1/1500
1/1 [==============================] - 1s 718ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 2/1500
1/1 [==============================] - 0s 33ms/step - loss: 0.7119 - accuracy: 0.5000
Epoch 3/1500
1/1 [==============================] - 0s 4ms/step - loss: 1.4958 - accuracy: 0.5000
Epoch 4/1500
1/1 [==============================] - 0s 11ms/step - loss: 6.0891 - accuracy: 0.5000
Epoch 5/1500
1/1 [==============================] - 0s 3ms/step - loss: 2.1110 - accuracy: 0.5000
Epoch 6/1500
1/1 [==============================] - 0s 4ms/step - loss: 5.0066 - accuracy: 0.5000
Epoch 7/1500
1/1 [==============================] - 0s 4ms/step - loss: 2.0258 - accuracy: 0.5000
Epoch 8/1500
1/1 [==============================] - 0s 5ms/step - loss: 4.0737 - accuracy: 0.5000
Epoch 9/1500
1/1 [==============================] - 0s 6ms/step - loss: 1.9577 - accuracy: 0.5000
Epoch 10/1500
1/1 [==============================] - 0s 5ms/step - loss: 3.2774 - accuracy: 0.5000
Epoch 11/1500
1

1/1 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.7500
Epoch 83/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.2500
Epoch 84/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7500
Epoch 85/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.2500
Epoch 86/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.6928 - accuracy: 0.7500
Epoch 87/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.2500
Epoch 88/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6928 - accuracy: 0.7500
Epoch 89/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6928 - accuracy: 0.2500
Epoch 90/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.7500
Epoch 91/1500
1/1 [==============================] - 0s 17ms/step - loss: 0.6928 - accuracy: 0.2500
Epoch 92/1500
1/1 [===

1/1 [==============================] - 0s 9ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 164/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 165/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 166/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 167/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 168/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 169/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 170/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 171/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 172/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.2500
Epoch 173/1500


1/1 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 245/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 246/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 247/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 248/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 249/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 250/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6917 - accuracy: 0.5000
Epoch 251/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.6917 - accuracy: 0.5000
Epoch 252/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.6917 - accuracy: 0.5000
Epoch 253/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6917 - accuracy: 0.5000
Epoch 254/150

1/1 [==============================] - 0s 15ms/step - loss: 0.6906 - accuracy: 0.7500
Epoch 326/1500
1/1 [==============================] - 0s 15ms/step - loss: 0.6905 - accuracy: 0.7500
Epoch 327/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.6905 - accuracy: 0.7500
Epoch 328/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.6905 - accuracy: 0.7500
Epoch 329/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.6905 - accuracy: 0.7500
Epoch 330/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.6904 - accuracy: 0.7500
Epoch 331/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.6904 - accuracy: 0.7500
Epoch 332/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.6904 - accuracy: 0.7500
Epoch 333/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.6904 - accuracy: 0.7500
Epoch 334/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.6903 - accuracy: 0.7500
Epoch 335/150

...

1/1 [==============================] - 0s 7ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 1135/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 1136/1500
1/1 [==============================] - 0s 16ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 1137/1500
1/1 [==============================] - 0s 14ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 1138/1500
1/1 [==============================] - 0s 20ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1139/1500
1/1 [==============================] - 0s 10ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1140/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1141/1500
1/1 [==============================] - 0s 7ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1142/1500
1/1 [==============================] - 0s 7ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1143/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.0056 - accuracy: 1.0000
Ep

1/1 [==============================] - 0s 6ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 1215/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 1216/1500
1/1 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 1217/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 1218/1500
1/1 [==============================] - 0s 7ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 1219/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 1220/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 1221/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 1222/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 1223/1500
1/1 [==============================] - 0s 4ms/st

1/1 [==============================] - 0s 15ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1295/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1296/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1297/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1298/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1299/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1300/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1301/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 1302/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 1303/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000
Epoc

1/1 [==============================] - 0s 4ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1376/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1377/1500
1/1 [==============================] - 0s 8ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1378/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1379/1500
1/1 [==============================] - 0s 15ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1380/1500
1/1 [==============================] - 0s 14ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1381/1500
1/1 [==============================] - 0s 7ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 1382/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 1383/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 1384/1500
1/1 [==============================] - 0s 5ms/step - loss: 0.0028 - accuracy: 1.0000
Epoc

1/1 [==============================] - 0s 9ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1456/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1457/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1458/1500
1/1 [==============================] - 0s 16ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1459/1500
1/1 [==============================] - 0s 6ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1460/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1461/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1462/1500
1/1 [==============================] - 0s 4ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1463/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 1464/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.0024 - accuracy: 1.0000
Ep

In [12]:
print("time %f.3" % (t2 - t1))

time 38.528587.3


In [13]:
model.predict(training_data)

array([[0.0015527 ],
       [0.99774015],
       [0.9978236 ],
       [0.00295168]], dtype=float32)

## Generate model.cpp

In [14]:
from tinymlgen import port
import os

In [15]:
path = "../pico-tflmicro/examples/xor"

In [16]:
c_code = port(model, optimize=False, pretty_print = True)

open(path + "/model.cpp", "w").write('#include "model.h"\n' + c_code)

2021-12-07 16:18:58.831630: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmpkkrtlkir/assets


2021-12-07 16:18:59.564250: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:345] Ignored output_format.
2021-12-07 16:18:59.564277: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:348] Ignored drop_control_dependency.


70399

<pre>







</pre>
## Save tflite file and code generation

In [17]:
### see https://github.com/eloquentarduino/tinymlgen

import re
import hexdump

def my_port(tflite_model, variable_name='model_data', pretty_print=True):
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    
    preamble = '''
#include "model.h"

// if having troubles with min/max, uncomment the following
// #undef min    
// #undef max
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif
'''
    return preamble + c


In [18]:
def get_filename(optimizer):
    optimezedStr = "_optimized" if optimizer else ""
    return path + "/model_%d_%d%s.tflite" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2, optimezedStr)

In [19]:
def save_tflite(model, optimizer=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimizer:
        optimizers = [tf.lite.Optimize.DEFAULT]        
        converter.optimizations = optimizers
        
        def representative_dataset():
            data = training_data
            yield [data.astype(np.float32)]
        
        converter.representative_dataset = representative_dataset

    filename = get_filename(optimizer)

    tflite_model = converter.convert()
    print("%s %d bytes" % (filename, len(tflite_model)))

    with open(filename, 'wb') as f:
      f.write(tflite_model)
    
    code = my_port(tflite_model)
    open(filename + ".cpp", "w").write(code)

In [20]:
save_tflite(model, optimizer=True)

INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmp41lcu4bl/assets


INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmp41lcu4bl/assets
2021-12-07 16:19:00.645106: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:345] Ignored output_format.
2021-12-07 16:19:00.645128: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:348] Ignored drop_control_dependency.


../pico-tflmicro/examples/xor/model_16_128_optimized.tflite 5480 bytes


fully_quantize: 0, inference_type: 6, input_inference_type: 0, output_inference_type: 0


In [21]:
save_tflite(model, optimizer=False)

INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmpodhd41zl/assets


INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmpodhd41zl/assets
2021-12-07 16:19:01.707636: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:345] Ignored output_format.
2021-12-07 16:19:01.707658: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:348] Ignored drop_control_dependency.


../pico-tflmicro/examples/xor/model_16_128.tflite 11352 bytes


## Predict tflite files

In [22]:
def predict_tflite(optimizer):
    filename = get_filename(optimizer)
    interpreter = tf.lite.Interpreter(model_path=filename)
    interpreter.allocate_tensors()

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    data = [[tf.cast(0.0, tf.float32), tf.cast(0.0, tf.float32)]]

    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    print("%s %f" % (filename, pred))

    
predict_tflite(optimizer=False)
predict_tflite(optimizer=True)

../pico-tflmicro/examples/xor/model_16_128.tflite 0.001553
../pico-tflmicro/examples/xor/model_16_128_optimized.tflite 0.000000
